# ASSIGMENT 1

## Lara Monteserín Placer

## María Ferrero Medina

## Introduction

The aim of this project is to design a machine learning model that is able to predict the energy produced by the Sotavento wind farm. For this purpose, a dataset with 555 features ans 4748 instances is available.

The structure of the study will be the following:

1. Exploratory Data Analysis
2. Methodology
3. KNN model
4. Decission tree model
5. Ensemble method 1 model
6. Ensemble method 2 model
7. Selection and performance of the final model

For each of the models created, several steps have been followed to optimize each model:

a) Hyperparameter tuning
b) Imputation techniques
c) Feature selection


Things to add: - another idea (new library?, new idea...)


## 1. Exploratory Data Analysis

Before starting to build the model, an EDA is made as a first approach to gain understanding of the dataset. In this Exploratory Data Analysis the data type of the features will be verified, the number of instances and features will be determined. Also, a brief summary of the missing values and columns with constant value will be included. 

### 1.1. Number of instances and features

This dataset has 4748 instances and 555 features.

### 1.2. Nature of the variables

This dataset contains information about the meteorological conditions in several locations, the time the measures of these conditions were made and the energy produced at each moment. 

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Read the data that is compressed as a gzip
wind_ava = pd.read_csv('wind_available.csv.gzip', compression="gzip")

# Display the first rows of the dataset just to see it
wind_ava.head()

# Display the data type of each column
column_data_types = wind_ava.dtypes
print(column_data_types)

energy     float64
year         int64
month        int64
day          int64
hour         int64
            ...   
v100.21    float64
v100.22    float64
v100.23    float64
v100.24    float64
v100.25    float64
Length: 555, dtype: object


After having checked the data types of all the different features, it has been verified that there are:

- 551 numerical variables (real numbers). From this 551, one is the energy, that is the output of the problem. And the remaining 550 are relative to the 22 different meteorological conditions measured at the 25 different locations.

- 4 numerical variables (integers). These 4 variables are the year, day, month and hour of the day. These variables characterize the moment the measures were taken.

### 1.3. Check for missing values 

In [2]:
# Return the number of Null values for each column
null_values = wind_ava.isnull().sum()
# Return the number of NaN values for each column (just in case they are not the same)
nan_values = wind_ava.isna().sum()

# Store in missing values the amount of Null and NaN values of each column
missing_values = pd.DataFrame({
    'Column': null_values.index,
    'Null Values': null_values.values,
    'NaN Values': nan_values.values
})

# Print the amount of Null and Nan values
print(missing_values)

# Identify columns with Null or NaN values
columns_with_null = wind_ava.columns[wind_ava.isnull().any()]
columns_with_nan = wind_ava.columns[wind_ava.isna().any()]

# Display the number of columns which have missing values
print("Number of columns with Null Values:", len(columns_with_null))
print("Number of columns with NaN Values:", len(columns_with_nan))

      Column  Null Values  NaN Values
0     energy            0           0
1       year            0           0
2      month            0           0
3        day            0           0
4       hour            0           0
..       ...          ...         ...
550  v100.21          261         261
551  v100.22          387         387
552  v100.23          569         569
553  v100.24          579         579
554  v100.25          436         436

[555 rows x 3 columns]
Number of columns with Null Values: 550
Number of columns with NaN Values: 550


All meteorological variables have missing values in different instances. The 4 categories that characterize the moment the measure was made and the target feature'energy' do not have missing values.


### 1.4. Check for constant columns


In [ ]:
# Check for constant values in each column
constant_columns = wind_ava.columns[wind_ava.nunique() == 1]

# Print columns with constant values
print("Columns with constant values:", constant_columns)


There are no constant columns. 


### 1.5. Type of problem

The objective of the model is to estimate the energy, as it is a continuous numerical value, this is a **regression problem**. 

## 2. Methodology

This section will explain the methodology that is going to be followed to evaluate the models. The evaluation tech Which are the outer evaluation and the inner evaluation. And the metrics.

- On the one hand, for the **inner evaluation**, crossvalidation will be the method applied. Crossvalidation will be used to determine which is the best combination of hyperparameters. 

- On the other hand, for the **outer evaluation**, holdout evaluation will be used. Thie method will be used to estimate the future performance of the designed method.

Later, to improve the performance of the method, once it is already computed the outer performance, the hyperparameters will be tuned again but this time using the whole dataset.

The objective function that is going to be used for the validation of the method is the Mean Squared Error (MSE). This metric is more sensitive to outliers and to distant values as it squares the magnitudes. This is useful to penalize the errors that are larger, and avoid having a model that might have such large errors. 

**It could be longer this explanation**

Note: as the variable that is going to be predicted is the 'energy', only the variables related to the meteorological characteristics will be used. It is considered that the energy produced does not depend on the moment of the day it is being produced.

## 3. KNN model

First, the simplest model is tried.

## 3. First approach to the models

The idea for this section is to have a first approach for both the KNN and the trees models. This will be done considering all features from the dataset and using the mean as the imputation method. Later on, this two aspects will be changed and it will be checked how that changes affect the model, but first, the models.

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score

# Split into train and test sets
X = wind_ava.drop('energy', axis=1)
y = wind_ava['energy']
# stratify = y (?)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 100514164)

# Define the first model as a Pipeline with a preprocessing stage with knn
# first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('feature_selection',SelectKBest(f_regression)),('regression',DecisionTreeRegressor())])
first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor())])
first_tree.fit(X_train,y_train)
y_predicted = first_tree.predict(X_test)
R_2 = r2_score(y_test,y_predicted)
print(R_2)

0.3797294092124536


In [35]:
from sklearn.neighbors import KNeighborsRegressor
# Now we are going to create a basic model but using KNN
# Define the first model as a Pipeline with a preprocessing stage with knn
# first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('feature_selection',SelectKBest(f_regression)),('regression',KNeighborsRegressor(n_neighbors=5))])
first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',KNeighborsRegressor(n_neighbors=31))])
first_tree.fit(X_train,y_train)
y_predicted = first_tree.predict(X_test)
R_2 = r2_score(y_test,y_predicted)
print(R_2)


0.17710274456865427


In [39]:
# Now we are going to perform feature selection by hand

# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)
sotavento = wind_ava.filter(regex='\.13$', axis=1)

# Split into train and test sets
X = sotavento
y = wind_ava['energy']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 100514164)

# Define the first model as a Pipeline with a preprocessing stage with knn
# first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('feature_selection',SelectKBest(f_regression)),('regression',DecisionTreeRegressor())])
first_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor())])
first_tree.fit(X_train,y_train)
y_predicted = first_tree.predict(X_test)
R_2_first = r2_score(y_test,y_predicted)

# SECOND OPTION - Selecting the features that correspond to magnitudes related to the wind
wind = wind_ava.filter(regex='^(u|v).*$', axis=1)
X = sotavento
y = wind_ava['energy']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 100514164)
# Define the first model as a Pipeline with a preprocessing stage with knn
second_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor())])
first_tree.fit(X_train,y_train)
y_predicted = first_tree.predict(X_test)
R_2_second = r2_score(y_test,y_predicted)

# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento
wind = wind_ava.filter(regex='^(u|v).*\.13$', axis=1)
X = sotavento
y = wind_ava['energy']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 100514164)
# Define the first model as a Pipeline with a preprocessing stage with knn
second_tree = Pipeline([('imputer',KNNImputer(n_neighbors=3)),('regression',DecisionTreeRegressor())])
first_tree.fit(X_train,y_train)
y_predicted = first_tree.predict(X_test)
R_2_third = r2_score(y_test,y_predicted)

print(R_2_first)
print(R_2_second)
print(R_2_third)

0.35045367458558485
0.3878773130396762
0.38420468177756084
